<a href="https://colab.research.google.com/github/ToshikiShimizu/PyTorch/blob/master/intro_dev/chapter4/4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torchvision

    100% |████████████████████████████████| 519.5MB 32kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a02a000 @  0x7f0ce895f2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 1.2MB/s 
    100% |████████████████████████████████| 2.0MB 15.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar xf 102flowers.tgz
!mkdir oxford-102
!mkdir oxford-102/jpg
!mv jpg/*.jpg oxford-102/jpg

--2018-11-17 05:32:42--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/x-gzip]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  11.6MB/s    in 31s     

2018-11-17 05:33:14 (10.7 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]



In [0]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
from torch import nn

In [0]:
img_data = ImageFolder("oxford-102/",transform=transforms.Compose([transforms.Resize(80),transforms.CenterCrop(64),transforms.ToTensor()]))
batch_size = 64
img_loader = DataLoader(img_data, batch_size = batch_size, shuffle=True)

In [0]:
from torch import nn

In [0]:
nz = 100
ngf = 32

class GNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.main = nn.Sequential(
    nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
    nn.BatchNorm2d(ngf*8),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf*2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf*2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf*2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
    nn.Tanh()
    )
  def forward(self, x):
    out = self.main(x)
    return out

In [0]:
ndf = 32
class DNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.main = nn.Sequential(
    nn.Conv2d(3, ndf, 4, 2, 1 ,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf*2),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf*4),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf*8),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
    )
  def forward(self, x):
    out = self.main(x)
    return out.squeeze()
        
        

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm

In [0]:
d = DNet().to("cuda:0")
g = GNet().to("cuda:0")
opt_d = optim.Adam(d.parameters(),
                  lr = 0.0002, betas=(0.5, 0.9999))
opt_g = optim.Adam(g.parameters(),lr = 0.0002, betas=(0.5, 0.999))

ones = torch.ones(batch_size).to("cuda:0")
zeros = torch.zeros(batch_size).to("cuda:0")
loss_f = nn.BCEWithLogitsLoss()
fixed_z = torch.randn(batch_size, nz, 1, 1).to("cuda:0")

In [0]:
from statistics import mean
def train_dcgan(g,d, opt_g, opt_d, loader):
  log_loss_g = []
  log_loss_d = []
  for real_img, _ in tqdm.tqdm(loader):
    batch_len = len(real_img)
    reak_img = real_img.to("cuda:0")
    z = torch.randn(batch_len, nz, 1, 1).to("cuda:0")
    fake_img = g(z)
    fake_img_tensor = fake_img.detach()
    out = d(fake_img)
    loss_g = loss_f(out, ones[:barch_len])
    log_loss_g.append(loss_g.item())
    d.zero_grad(), g.zero_grad()
    loss_g.backward()
    opt_g.step()
    
    real_out = d(real_img)
    loss_d_real = loss_f(real_out, ones[:batch_len])
    fake_img = fake_img_tensor
    fake_out = d(fake_img_tensor)
    loss_d_fake = loss_f(fake_out, zeros[:batch_len])
    loss_d = loss_d_real + loss_d_fake
    log_loss_d.append(loss_d.item())
    d.zero_grad(), g.zero_grad()
    loss_d.backward()
    opt_d.step()
  return mean(log_loss_g), mean(log_loss_d)

In [0]:
!mkdir out

In [16]:
for epoch in range(300):
  train_dcgan(g,d,opt_g,opt_d,img_loader)
  if epoch % 10 == 0:
    torch.save(
    g.state_dict(),
    "out/g_{:03d}.prm".format(epoch),pickle_protocol=4)
    torch.save(
    d.state_dict(),
    "out/g_{:03d}.prm".format(epoch),pickle_protocol=4)
    generated_img = g(fixed_z)
    save_image(generated_img,"out/{:03d}.jpg".format(epoch))

  0%|          | 0/128 [00:00<?, ?it/s]

AttributeError: ignored